In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-01-03 06:40:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-01-03 06:40:09 (191 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
user_rating_counts = df_ratings['user'].value_counts()
book_rating_counts = df_ratings['isbn'].value_counts()
valid_user_indice = user_rating_counts[user_rating_counts>=200].index
valid_book_indice = book_rating_counts[book_rating_counts>=100].index

filtered_df_ratings = df_ratings[(df_ratings['user'].isin(valid_user_indice))
                                  &
                                 (df_ratings['isbn'].isin(valid_book_indice))]

filtered_df_books = df_books[df_books['isbn'].isin(filtered_df_ratings['isbn'])]
df=filtered_df_books.merge(filtered_df_ratings, on='isbn')
filtered_df = df.drop_duplicates(subset=['title','user'], keep='first')
piv = filtered_df.pivot(index='title', columns='user', values='rating').fillna(0)

In [5]:
titles = list(piv.index.values)
X = piv.values

def title_2_index(title):
  ind = titles.index(title)  # The row index corresponding to the book title
  return X[ind,:]            # Returns the ratings of all users for this book

def index_2_title(ind):
  return titles[ind]         # Return book title based on index

In [6]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

    nn = NearestNeighbors(metric="cosine", algorithm="brute")
    nn.fit(X)
    row = title_2_index(book)    # all users' ratings for this book
    dist, inds = nn.kneighbors(np.reshape(row,[1,-1]), len(titles), True)

    recommended_distances=dist[0].tolist()
    rec_index=inds[0].tolist()
    recommended_titles =  [titles[i] for i in rec_index]

    recommended_books = []
    recommended_books.append(recommended_titles[0])
    recommended_books.append([ [i,j] for i,j in zip(recommended_titles[1:], recommended_distances[1:])])

    updated_recommended_books = []
    updated_recommended_books.append(recommended_titles[0])
    updated_recommended_books.append(recommended_books[1][:5][::-1])

    recommended_books=updated_recommended_books

    return recommended_books

In [7]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
